# Zadání

Na dodaném zvukovém souboru proveďte následující body. Postup a získané výsledky prezentujte krátkou zprávou se spoustou obrázků (ukázky spekter signálu před a po filtraci, spektrální charakteristiky filtrů apod.)

1. **Návrh a vyzkoušení filtrů:**  
   Navrhněte a vyzkoušejte několik variant FIR a IIR filtrů (např. různé řády modelu, různé okénkové funkce, …) pro odstranění parazitní frekvence. Diskutujte vliv řádu a typu filtru na výsledné zatlumení parazitní frekvence (zkuste alespoň jeden FIR filtr spočítat podle postupu z přednášek; ostatní můžete navrhnout přímo např. v Matlabu, Pythonu, …).

2. **Převzorkování signálu:**  
   Převeďte signál na vzorkovací frekvenci 8 kHz.
   - *Pozor na aliasing efekt (nutnost frekvenčního omezení na 4 kHz).*

3. **Odstranění aditivního šumu:**  
   Zkuste z převzorkované nahrávky odstranit aditivní šum pomocí metody spektrálního odečtu:
   - Rozdělte signál na nepřekrývající se krátké úseky (segmenty) délky *m* – ideálně.
   - Spočítejte DFT pro každý segment (vlastním algoritmem FFT).
   - Odhadněte amplitudové spektrum aditivního šumu (na neřečových segmentech).
   - Aplikujte algoritmus odstranění aditivního šumu (přes všechny segmenty).
   - Rekonstruujte řečový signál pomocí inverzní FFT (nezapomeňte na fázi, viz obrázek).
   - Diskutujte různé velikosti FFT, různé váhy odečtu *α*, a možnosti odhadu šumového spektra (např. bodový odhad, průměr přes celou nahrávku, klouzavý průměr, …).

Úlohu můžete řešit v libovolném jazyce. Pokud ji budete řešit v Matlabu či Pythonu (vřele doporučuji), alespoň FFT a jeden FIR filtr si zkuste udělat vlastní.


# Vypracování

## 1) FIR a IIR filtry


Nejdříve je potřeba naimportovat všechny důležité knihovny a vlastní funkce

In [ ]:
from scipy.io import wavfile
from scipy.fft import fft
from scipy.signal import butter, filtfilt, resample, spectrogram, freqz, firwin, cheby1, cheby2, ellip, sosfiltfilt
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
import utils_functions 

### a) Nalezení parazitní frekvence

In [ ]:
fs, data = wavfile.read('veta.wav')

print(f"Vzorkovací frekvence: {fs} Hz")

In [ ]:
#přehrání nahrávky před filtrací
data_float = data.astype(np.float32)
Audio(data=data,rate=fs)

In [ ]:
# vykreslení spektrogramu pro obdržený audio záznam
utils_functions.plot_spectrogram(data,fs)

Již z tohoto grafu je velmi patrné, že parazitní signál je znázorněn vodorvonou přímkou, na frekvenci mezi 5000 Hz a 7500 Hz.

Následuje vykreslení amplitudové frekvenční charakteristiky pro audio záznam. Parazitní frekvence je i zde velmi dobře vidět, protože se projevuje velmi vysokou hodnoty amplitudy. 

In [ ]:
utils_functions.plot_amplitude_char(data,fs)

Nalezení parazitní frekvence je provedeno pomocí převodu na frekvenční spektrum a následné nalezení, pro jakou frekvenci je amplituda největší.

In [ ]:
freqs, amp = utils_functions.compute_fourier(data,fs)

parasite_freq_index = np.argmax(amp)
parasite_freq = freqs[parasite_freq_index]
print("parazitni frekvence: ",parasite_freq)

Parazitní frekvence je tedy nalezena jako f_parazitni= 5499.993 Hz.

### b) FIR filtry
V první části jsou navrženy FIR filtry pomocí Python knihoven. Všechny filtry jsou navrženy jako pásmová zádrž kolem nalezené parazitní frekvence, což vede na její potlačení. 

Nejdříve je demonstrován vliv volby okénkové funkce na amplitudovou charakteristiku samotného filtru, následně je filtr aplikován na signál ze zadání a je vykreslen spektrogram pro odfiltrovaný signál. 

Pro každý filtr je pak znázorněno i porovnání amplitudové charakteristiky před filtrací a po filtraci.


Volba řádu filtru a šířky pásma byla určena experimentálně. Nízké řády (řád < 100) vykazují v daném kontextu příliš široké přechodové pásmo – útlum parazitní frekvence tak není dostatečný. Pro řády vyšší je naopak rozdíl v potlačení vyšší, ale vliv na zadanou nahrávku je pro vysoké řády zanedbatelný a není tak důvod zesilovat útlum více, než je požadováno. 
Proto byl řád zvolen jako 255. Co se týče velikost zádržného pásma, tak to je voleno tak, aby filtr dokázal potlačit parazitní frekvenci a ovlivnil co nejméně vedlejší řečový signál. Nízké pásmo by mohlo vést k neuplnému potlačení parazitního signálu, naopak příliš široké pásmo by způsobilo, že by došlo k potlačení i frekvencí, kde se vyskytuje pouze řečový signál. 

In [ ]:
# definice hranic pro pásmovou zádrž
band_width = 300
original_data = data.copy()
dolni_mez = parasite_freq - band_width
horni_mez = parasite_freq + band_width

filter_order = 255 # zvolený pevný řád filtru pro testování vlivu různých okénkových funkcí

#### b.1) Okénková funkce Hann

In [ ]:
fir_coeff = firwin(filter_order,[dolni_mez, horni_mez],window="hann",pass_zero=True, fs=fs)

filtered_data = filtfilt(fir_coeff,[1.0],data)

utils_functions.plot_graph_FIR(fir_coeff,filtered_data, original_data, fs,"okénková funkce Hann, řád filtru = "+str(filter_order))

Z grafu spektrogramu je vidět, že aplikace FIR filtru spolu s Hannovo okénkovou funkcí způsobila, že amplituda na parazitní frekvenci velmi silně poklesla. Toto tvrzení podporuje i graf amplitudové frekvenční charakteristiky, kde se signál před filtrací a po filtraci téměř perfektně překrývá, kromě právě okolí parazitní frekvence, kde je amplituda signálu velmi silně potlačena.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

#### b.2) Okénková funkce Hamming

In [ ]:

fir_coeff = firwin(filter_order,[dolni_mez, horni_mez],window="hamming",pass_zero=True, fs=fs) 

filtered_data = filtfilt(fir_coeff,[1.0],data)

utils_functions.plot_graph_FIR(fir_coeff,filtered_data,original_data,fs,"okénková funkce Hamming, řád filtru = "+str(filter_order))

Výsledky z grafů jsou velmi podobné předchozímu příkladu. Zásadně se liší především amplitudová charakteristika samotného filtru.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

#### b.3) Okénková funkce Blackman

In [ ]:
fir_coeff = firwin(filter_order,[dolni_mez, horni_mez],window="blackman",pass_zero=True, fs=fs)

filtered_data = filtfilt(fir_coeff,[1.0],data)

utils_functions.plot_graph_FIR(fir_coeff,filtered_data,original_data,fs,"okénková funkce Blackman, řád filtru = "+str(filter_order))

Znovu jsou dosažené výsledky velmi podobné jako u ostatních okénkových funkcí. Zásadně se liší především amplitudová charakteristika samotného filtru.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

#### b.4) Okénková funkce Pravoúhlé okénko

In [ ]:

fir_coeff = firwin(filter_order,[dolni_mez, horni_mez],window="boxcar",pass_zero=True, fs=fs)

filtered_data = filtfilt(fir_coeff,[1.0],data)

utils_functions.plot_graph_FIR(fir_coeff,filtered_data,original_data, fs,"okénková funkce pravoúhlé okénko, řád filtru = "+str(filter_order))


Znovu jsou dosažené výsledky velmi podobné jako u ostatních okénkových funkcí. Zásadně se liší především amplitudová charakteristika samotného filtru, která je velmi kmitavá, což je očekávané. Volbou vyššího řádu filtru by překmity zůstaly stejné, ale snižovalo by se přechodové pásmo.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

#### b.5) Okénková funkce Hamming s nižším řádem
V této části je znázorněn vliv řádu filtru na úspěšnost potlačení ideálně pouze parazitního signálu, tedy parazitní frekvence.

In [ ]:
filter_order = 55 # zvolený řád je 55

fir_coeff = firwin(filter_order,[dolni_mez, horni_mez],window="hamming",pass_zero=True, fs=fs)

filtered_data = filtfilt(fir_coeff,[1.0],data)

utils_functions.plot_graph_FIR(fir_coeff,filtered_data,original_data,fs,"okénková funkce Hamming, řád filtru = "+str(filter_order))

Na těchto grafech lze pozorovat, že volbou nízkého řádu je přechodové pásmo filtru velmi velké a amplituda samotného filtru je velmi nízká. Takovýto řád filtru tedy není schopen potlačit působení celého parazitního signálu, ale dokáže jej pouze mírně zeslabit. To je ostatně velmi zřejmé na spektrogramu a amplitudové frekvenční charakteristice.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

#### b.6) Vlastní implementace FIR filtru
V této části je demonstrována implementace vlastního FIR filtru podle přednášek. Řád je zvolen znovu jako 255. Vykresleny jsou všechny iterace.  

In [ ]:
filter_order = 255
filtered_data,fir_coeff = utils_functions.custom_FIR(filter_order, 300, parasite_freq, data, fs, plot=False)
utils_functions.plot_graph_FIR(fir_coeff,filtered_data,original_data,fs,"Vlastní implementace filtru, řád filtru = "+str(filter_order))

Na grafech lze vidět, že i vlastní implementace FIR filtru podle přednášek dokáže efektivně potlačit parazitní frekvenci.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

## c) IIR filtry

V této části je nejprve demonstrován vliv různých IIR filtrů a následně i vliv různých řádů filtrů. Výsledky jsou demonstrovány stejným způsobem, jako tomu bylo u FIR filtrů.

Nejprve je potřeba opět stanovit pásmo pásmové zádrže. Tentokrát je však potřeba meze znormovat.

Co se týká volby šířky pásmové zádrže, tak není důvod volit jej jinak, než tomu bylo u FIR filtrů. Řád u IIR filtrů se chová jinak, protože většina použitých filtrů poskytuje dostatečně ostré přechody a útlum pro potlačení daného parazitního signálu, tedy lze použít řád filtru 1. 

In [ ]:
band_width = 300
original_data = data
dolni_mez = parasite_freq - band_width
horni_mez = parasite_freq + band_width

nyq_freq = 0.5*fs
dolni_mez = dolni_mez/nyq_freq
horni_mez = horni_mez/nyq_freq

filter_order = 1

#### c.1) Butterworthův IIR filtr

In [ ]:
b,a = butter(filter_order,[dolni_mez, horni_mez],btype="bandstop")

filtered_data = filtfilt(b,a,original_data)

utils_functions.plot_graph_IIR(b, a, filtered_data, original_data, fs, "Butterworthův filtr - pásmová zádrž")

Z grafů je zřejmé, že Butterworthův filtr dokázal úspěšně potlačit parazitní frekvenci, jak je názorné na spektrogramu a amplitudové frekvenční charakteristice před filtrací a po filtraci.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

#### c.2) Chebyševův IIR filtr

In [ ]:
rp = 0.4
b,a = cheby1(filter_order,rp,[dolni_mez, horni_mez],btype="bandstop")
filtered_data = filtfilt(b,a,original_data)

utils_functions.plot_graph_IIR(b, a, filtered_data, original_data, fs, "Chebyševův filtr - pásmová zádrž")

Čebyševův filtr dosahuje velmi podobných výsledků jako Butterworthův s tím rozdílem, že mají jinou frekvenční charakteristiku samotného filtru.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

#### c.3) Chebyševův IIR filtr
uvažovaný řád filtru je 1.

In [ ]:
rs = 20
b,a = cheby2(filter_order,rs,[dolni_mez, horni_mez],btype="bandstop")
filtered_data = filtfilt(b,a,original_data)

utils_functions.plot_graph_IIR(b, a, filtered_data, original_data, fs, "Chebyševův filtr - pásmová zádrž")

Druhá varianta Čebyševova filtru dosahuje velmi odlišných výsledků oproti jeho první variantě. Je zřejmé, že pro 1. řád je přechodové pásmo samotného filtru příliš široké a způsobí tak potlačení i na frekvencích, kde se nevyskytuje parazitní signál, ale pouze řečový signál. To je velmi názorné na výsledné nahrávce.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

Pro demonstraci správné funkce Chebyševova filtru je provedena filtrace znovu, ale tentokrát pro řád 2.

In [ ]:
rs = 20
filter_order = 2 # nový řád filtru
b,a = cheby2(filter_order,rs,[dolni_mez, horni_mez],btype="bandstop")
filtered_data = filtfilt(b,a,original_data)

utils_functions.plot_graph_IIR(b, a, filtered_data, original_data, fs, "Chebyševův filtr - pásmová zádrž")

Nyní po zvýšení řádu je přechodové pásmo mnohonásobně užší, což způsobuje i útlum na na parazitní frekvenci a jejím blízkém okolí a vliv na řečový signál je tak menší.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

#### c.4) Cauerův IIR filtr

In [ ]:
filter_order =1 
b,a = ellip(filter_order,rp,rs,[dolni_mez,horni_mez],btype="bandstop")
filtered_data = filtfilt(b,a,original_data)
utils_functions.plot_graph_IIR(b, a, filtered_data, original_data, fs, "Cauerův filtr - pásmová zádrž")

In [ ]:
#přehrání nahrávky po filtraci 
data_float = filtered_data.astype(np.float32)
Audio(data=filtered_data,rate=fs)

#### c.5) Butterworthův IIR filtr pro různé řády
V této části je demonstrován vliv řádu filtru na kvalitu filtrace parazitní frekvence. Konkrétně budou demonstrovány řády 1, 10 a 100. 

In [ ]:
#butterworth pro ruze rady
filter_order_list = [1, 10, 100]


for order in filter_order_list:
    sos = butter(order, [dolni_mez, horni_mez], btype="bandstop", output='sos')
    
    filtered_data = sosfiltfilt(sos, original_data)
    
    if order==1:
        result_filtered_data = filtered_data # pro dalsi zpracovani
    utils_functions.plot_graph_IIR_sos(sos, filtered_data, original_data, fs,
                       "Butterworth Bandstop Filter, order = " + str(order))

Z grafů lze vidět, že zvýšením řádu se snižuje přechodové pásmo filtru a jeho amplitudová frekvenční charakteristika je ostřejší a dosahuje větší amplitudy. To však může způsobit potlačení nejen parazitního signálu.

In [ ]:
#přehrání nahrávky po filtraci 
data_float = result_filtered_data.astype(np.float32)
Audio(data=result_filtered_data,rate=fs)

# 2. ukol - převzorkování signálu na 8kHz
Pro převzorkování signálu je nejprve nutné použít filtr typu dolní propust na hodnotu menší než 4kHz. To je dáno Nyquistovu-Shanonnovu teorému, který říká, že vzorkovací frekvence musí být nejméně 2x větší než frekvence obsažena v signálu. 

In [ ]:
#Použití vyfiltrovaných dat 

target_cutoff = 3800 # frekvence pro low pass filtr
lowpass_order = 6 # řád filtru
b_low, a_low = butter(lowpass_order, target_cutoff / (0.5 * fs), btype='low')
filtered_low = filtfilt(b_low, a_low, result_filtered_data)

t_origin = np.arange(len(filtered_low)) / fs

fs_resample = 8000
t_new = np.arange(len(filtered_low)) / fs_resample
t_new_valid = np.array([t for t in t_new if t <= t_origin[-1]])
resampled_data = np.interp(t_new_valid, t_origin, filtered_low)

print("Nová vzorkovací frekvence:", fs_resample, "Hz")

plt.figure(figsize=(12, 6))
plt.plot(t_origin, filtered_low, label='Původní signál', linewidth=1)
plt.plot(t_new_valid, resampled_data, 'r-', label='Převzorkovaný signál (interp)', linewidth=1)
plt.xlabel("Čas (s)")
plt.ylabel("Amplituda")
plt.title("Porovnání původního a převzorkovaného signálu (interp)")
plt.legend()
plt.grid(True)
plt.show()

# ověření pomocí funkce resample

num_samples_new = int(np.round(len(filtered_low) * fs_resample / fs))
resampled_data_func = resample(filtered_low, num_samples_new)
t_resampled = np.arange(len(resampled_data_func)) / fs_resample

plt.figure(figsize=(12, 6))
plt.plot(t_origin, filtered_low, label='Původní signál', linewidth=1)
plt.plot(t_resampled, resampled_data_func, 'r-', label='Převzorkovaný signál (resample)', linewidth=1)
plt.xlabel("Čas (s)")
plt.ylabel("Amplituda")
plt.title("Porovnání původního a převzorkovaného signálu (resample)")
plt.legend()
plt.grid(True)
plt.show()

utils_functions.plot_spectrogram(resampled_data,fs_resample)

freqs, amp=utils_functions.compute_fourier(resampled_data,fs_resample)
plt.figure(figsize=(10, 6))
plt.plot(freqs, amp, linewidth=1)
plt.title('Amplitude-Frequency Characteristic')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()


Z grafů je velmi dobře vidět, že převzorkováním signálu nebyla ztracena téměř žádná informace, jelikož původní a převzorkované signály jsou téměř identické v amplitudě.  

# 3) Odstranění aditivního šumu
Pro odstranění aditivního šumu bude použita metoda spektrálního odečtu. 

Charakteristiky šumu neznáme, tedy je nutné je odhadnout. 

In [ ]:
# hrubý odhad částí nahrávky, kdy je aktivní pouze šum
start_noise1 = 0
end_noise1 = 2 
start_noise2 = 8
end_noise2 = len(resampled_data) / fs  

idx_start_noise1 = int(start_noise1 * fs)
idx_end_noise1   = int(end_noise1 * fs)
idx_start_noise2 = int(start_noise2 * fs)
idx_end_noise2   = int(end_noise2 * fs)

noise_segment1 = resampled_data[idx_start_noise1:idx_end_noise1]
noise_segment2 = resampled_data[idx_start_noise2:idx_end_noise2]

noise = np.concatenate((noise_segment1, noise_segment2))

t_noise = np.arange(len(noise)) / fs

plt.figure(figsize=(12, 4))
plt.plot(t_noise, noise, label="Vybrané segmenty šumu", linewidth=1)
plt.xlabel("Čas (s)")
plt.ylabel("Amplituda")
plt.title("Extrahované segmenty šumu (0–2 s a 8 s až do konce)")
plt.legend()
plt.grid(True)
plt.show()

utils_functions.plot_spectrogram(noise, fs)

V následující části bude aplikována metoda spektrálního odečtu spolu s demonstrováním vlivu délky segmentu či míry spektrálního odečítání. Uvnitř funkce noise_cancel dochází k rekonstrukci signálu pomocí inverzní Fourierovy transformace.

In [ ]:
m = 2**8 # délka segmentu
alpha = 0.5 # míra spektrálního odečtu
clean_data = utils_functions.noise_cancel(m,noise,resampled_data,alpha)

utils_functions.plot_spectrogram(clean_data,fs_resample)

clean_data_float = clean_data.astype(np.float32)
Audio(data=clean_data_float,rate=fs_resample)

Toto nastavení se subjektivně jeví jako nejlepší, jelikož se šum velmi výrazně potlačen a řečová část se zdá býti téměř nepoškozena, jen mírně potlačena.

In [ ]:
m = 2**12
alpha = 0.5

clean_data = utils_functions.noise_cancel(m,noise,resampled_data,alpha)

utils_functions.plot_spectrogram(clean_data,fs_resample)

clean_data_float = clean_data.astype(np.float32)
Audio(data=clean_data_float,rate=fs_resample)

In [ ]:
m = 2**12
alpha = 0.9

clean_data = utils_functions.noise_cancel(m,noise,resampled_data,alpha)

utils_functions.plot_spectrogram(clean_data,fs_resample)

clean_data_float = clean_data.astype(np.float32)
Audio(data=clean_data_float,rate=fs_resample)

Pro toto konkrétní nastavení je velmi dobře slyšet negativní vliv parametru míry potlačení šumu na řečový signál.

In [ ]:
m = 2**12
alpha = 0.2

clean_data = utils_functions.noise_cancel(m,noise,resampled_data,alpha)

utils_functions.plot_spectrogram(clean_data,fs_resample)

clean_data_float = clean_data.astype(np.float32)
Audio(data=clean_data_float,rate=fs_resample)

Jelikož je zde parametr míry potlačení šumu velmi nízký, tak ve výstupní nahrávce lze slyšet stále velmi výrazný vliv šumu.

In [ ]:
m = 2**11
alpha = 0.6

clean_data = utils_functions.noise_cancel(m,noise,resampled_data,alpha)

utils_functions.plot_spectrogram(clean_data,fs_resample)

clean_data_float = clean_data.astype(np.float32)
Audio(data=clean_data_float,rate=fs_resample)

# Závěr
Tatp semestrální práce se zabývala zpracováním řečového signálu, který byl ovlivněn parazitní frekvencí a šumem. Cílem práce bylo odstranit parazitní frekvenci, navrhnout a aplikovat různé typy filtrů, převzorkovat signál a odstranit aditivní šum.

V první části se práce zaměřila především na odstranění parazitní frekvence z řečového signálu. Pro odstranění bylo aplikováno několik různých FIR a IIR filtrů s různými parametry. Téměř ve všech případech došlo k úspěšnému odstranění parazitní frekvence. Také byl implementován vlastní FIR filtr, který však musel být aplikován kaskádně, aby byly výsledky dostatečné.

Ve druhé části práce bylo provedeno převzorkování signálu na frekvenci 8 kHz. Před samotným převzorkováním byl aplikován dolní propust filtr, aby byl splněn Nyquistův-Shannonův teorém a zabráněno aliasingu. Pro převzorkování byla využita lineární interpolace.

Třetí část práce se zaměřila na odstranění aditivního šumu. Byla provedena analýza šumu a odhad jeho charakteristik. Následně byla aplikována metoda spektrálního odečtu pro odstranění šumu. Byly získány různé výsledky pro různé délky segmentů a hodnoty parametrů. Z výsledků lze pozorovat, že filtrace není příliš úspěšná, tedy hluk v pozadí se stále slyšet. Lepších výsledků by bylo teoreticky možné dosáhnout pomocí složitějších metod nebo lepším odhadem charakteristik šumu.

Závěrem lze shrnout, že cíle práce byly splněny. Odstranění parazitní frekvence a převzorkování signálu bylo úspěšné. Z výsledků lze pozorovat, že při přiliš vysoké hodnotě míry potlačení šumu dojde k narušení řečového signálu na úkor potlačení šumu. Naopak volba nižší hodnoty vede na menší potlačení šumu, avšak řečová část zůstává neporušena. Odstranění šumu však vyžaduje další zlepšení metod a technik. 